In [1]:
import pandas as pd
tab = pd.read_excel('/data/EGD/metadata/Segmentation_source.xlsx')

In [3]:
len(tab[tab.Observer != 'AUTO'])

374

In [ ]:
beta = 0.5 #The interpolation parameter    
params1 = model1.named_parameters()
params2 = model2.named_parameters()

dict_params2 = dict(params2)

for name1, param1 in params1:
    if name1 in dict_params2:
        dict_params2[name1].data.copy_(beta*param1.data + (1-beta)*dict_params2[name1].data)

model.load_state_dict(dict_params2)

In [ ]:
# Setup
modelA = nn.Linear(1, 1)
modelB = nn.Linear(1, 1)

sdA = modelA.state_dict()
sdB = modelB.state_dict()

# Average all parameters
for key in sdA:
    sdB[key] = (sdB[key] + sdA[key]) / 2.

# stteRecreate model and load averaged state_dict (or use modelA/B)
model = nn.Linear(1, 1)
model.load_state_dict(sdB)

In [2]:
from nnunet.nn_unet import NNUnet
from utils.args import get_main_args

In [19]:
!export CUDA_VISIBLE_DEVICES=1 && python main_weights_mean.py --exec_mode predict --results /home/polina/DeepLearningExamples/PyTorch/Segmentation/nnUNet/results --brats --deep_supervision --depth 6 --filters 64 96 128 192 256 384 512 --min_fmap 2 --scheduler --learning_rate 0.0003 --epochs 150 --nfolds 3 --fold 0 --amp --gpus 1 --task 11 --save_ckpt

Global seed set to 1
Filters: [64, 96, 128, 192, 256, 384, 512],
Kernels: [[3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3]]
Strides: [[1, 1, 1], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2]]
Traceback (most recent call last):
  File "main_weights_mean.py", line 69, in <module>
    model = NNUnet(args).load_from_checkpoint(ckpt_path)
  File "/opt/conda/lib/python3.8/site-packages/pytorch_lightning/core/saving.py", line 157, in load_from_checkpoint
    model = cls._load_model_state(checkpoint, strict=strict, **kwargs)
  File "/opt/conda/lib/python3.8/site-packages/pytorch_lightning/core/saving.py", line 199, in _load_model_state
    model = cls(**_cls_kwargs)
TypeError: __init__() missing 1 required positional argument: 'args'


In [4]:
import torch
ckpt_path1 = '/home/polina/DeepLearningExamples/PyTorch/Segmentation/nnUNet/results/3_fold_brats_2021/fold-0/checkpoints/epoch=135-dice_mean=90.62.ckpt'
torch.cuda.set_device(1)
# torch.load(ckpt_path1)

In [6]:
ch = torch.load(ckpt_path1)

In [16]:
ch['state_dict'] = average_state_dict

In [18]:
torch.save(ch, '/home/polina/DeepLearningExamples/PyTorch/Segmentation/nnUNet/results/3_fold_brats_2021/avarage_weights_all_dicts.ckpt')

In [21]:
!cd /results/brats_infer && mkdir average

In [ ]:
!export CUDA_VISIBLE_DEVICES=0 && python main.py --exec_mode predict --task 11 --data /data/11_3d --brats --dim 3 --fold 0 --nfolds 3 --ckpt_path /home/polina/DeepLearningExamples/PyTorch/Segmentation/nnUNet/results/3_fold_brats_2021/avarage_weights_all_dicts.ckpt --results /results/brats_infer/average --amp --tta --save_preds

Global seed set to 1
0 training, 0 validation, 1251 test examples
Filters: [64, 96, 128, 192, 256, 384, 512],
Kernels: [[3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3]]
Strides: [[1, 1, 1], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2]]
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Testing:  49%|█████████████▎             | 616/1251 [2:01:05<1:57:14, 11.08s/it]